In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, auc, classification_report, roc_curve
import warnings
warnings.filterwarnings('ignore')

In [7]:
df=pd.read_csv("D:/cdrive/desktop/DIVYA/CAREER/guvi/PROJECT/CREDIT_CARD/creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
'''
This data is about the credit card fraud transactions.
Class column having 0 indicates a normal transaction.
Class column having 1 indicates a fraud transaction.
'''

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [11]:
# Getting the shape of the data
print('Shape :', df.shape)
print('Rows :', df.shape[0])
print('Columns :', df.shape[1])

Shape : (284807, 31)
Rows : 284807
Columns : 31


In [13]:
# Getting an idea about duplicate data
df.duplicated().sum()

1081

In [15]:
# Removing the duplicate rows
df.drop_duplicates(inplace = True)

In [17]:
# Getting an idea about duplicate data
df.duplicated().sum()

0

In [19]:
# Getting an idea about null values
df.isnull().sum().any()

False

In [21]:
# Normalizing the amount column
df['normAmount'] = StandardScaler().fit_transform(np.array(df['Amount']).reshape(-1, 1))

# Dropping the columns
df.drop(['Time', 'Amount'], inplace = True, axis = 1)

In [23]:
# Getting the proportion of values from 'Class' column
df['Class'].value_counts(normalize = True).round(4)

Class
0    0.9983
1    0.0017
Name: proportion, dtype: float64

In [ ]:
# We have 99.83% of the data from Class 0
# We have 0.17% of the data from Class 1

In [ ]:
# The above data indicates that there is a huge amount of imbaalance in the data

In [25]:
# Seperating input features and target variable
X = df.drop('Class', axis = 1)
y = df['Class']

# Performing train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5)

In [27]:
# Creating a Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=55)

# Fitting the model
clf.fit(X_train, y_train)

# Making predictions
y_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)

# Getting probability predictions for AUC-ROC calculation
y_prob_test = clf.predict_proba(X_test)[:, 1]  # Probabilities for the positive class (1)
y_prob_train = clf.predict_proba(X_train)[:, 1]

# Model Evaluation - Testing Dataset
print('Testing Accuracy :', np.round(metrics.accuracy_score(y_test, y_pred), 2))
print('Testing Precision :', np.round(metrics.precision_score(y_test, y_pred), 2))
print('Testing Recall :', np.round(metrics.recall_score(y_test, y_pred), 2))
print('Testing F1 Score :', np.round(metrics.f1_score(y_test, y_pred), 2))
print('Testing AUC-ROC Score :', np.round(metrics.roc_auc_score(y_test, y_prob_test), 2))

# Model Evaluation - Training Dataset
print('\nTraining Accuracy :', np.round(metrics.accuracy_score(y_train, y_train_pred), 2))
print('Training Precision :', np.round(metrics.precision_score(y_train, y_train_pred), 2))
print('Training Recall :', np.round(metrics.recall_score(y_train, y_train_pred), 2))
print('Training F1 Score :', np.round(metrics.f1_score(y_train, y_train_pred), 2))
print('Training AUC-ROC Score :', np.round(metrics.roc_auc_score(y_train, y_prob_train), 2))

# Classification Report
print("\n Classification Report:\n", metrics.classification_report(y_test, y_pred))


Testing Accuracy : 1.0
Testing Precision : 0.77
Testing Recall : 0.78
Testing F1 Score : 0.78
Testing AUC-ROC Score : 0.89

Training Accuracy : 1.0
Training Precision : 1.0
Training Recall : 1.0
Training F1 Score : 1.0
Training AUC-ROC Score : 1.0

 Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56654
           1       0.77      0.78      0.78        92

    accuracy                           1.00     56746
   macro avg       0.89      0.89      0.89     56746
weighted avg       1.00      1.00      1.00     56746



In [ ]:
'''
The model achieves 100% accuracy on both training and test data, suggesting overfitting, 
meaning it may have memorized patterns rather than generalizing well. 
While it performs well overall, its precision (0.77) means 23% of the predicted fraud cases were actually non-fraud (false positives). 
The recall (0.78) indicates that 22% of actual fraud cases were missed (false negatives), which can be risky in fraud detection.
The high weighted average (1.00) is due to the overwhelming number of non-fraud cases (Class 0), making the overall performance appear perfect.
To improve, we can focus on reducing overfitting and increasing recall to capture more fraud cases.
'''

In [27]:
# Using SMOTE to handle imbalance in the data
print('Before SMOTE, count of label 1 :', sum(y_train == 1))
print('Before SMOTE, count of label 0 :', sum(y_train == 0))

# SMOTE
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 5)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print('\nAfter SMOTE, count of label 1 :', sum(y_train_res == 1))
print('After SMOTE, count of label 0 :', sum(y_train_res == 0))

Before SMOTE, count of label 1 : 381
Before SMOTE, count of label 0 : 226599

After SMOTE, count of label 1 : 226599
After SMOTE, count of label 0 : 226599


In [31]:
# Hyperparameter Tuning using RandomizedSearchCV
param_dist = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10]
}

random_search = RandomizedSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

random_search.fit(X_train_res, y_train_res)

# Get best parameters
best_params = random_search.best_params_
print("\nBest Hyperparameters:", best_params)

# Train Optimized Model
model = DecisionTreeClassifier(**best_params, random_state=42)
model.fit(X_train_res, y_train_res)

# Predictions for Training Set
y_train_pred = model.predict(X_train_res)
y_train_prob = model.predict_proba(X_train_res)[:, 1]

# Predictions for Testing Set
y_test_pred = model.predict(X_test)
y_test_prob = model.predict_proba(X_test)[:, 1]

# Evaluate Model Performance - Training Set
train_accuracy = accuracy_score(y_train_res, y_train_pred)
train_roc_auc = roc_auc_score(y_train_res, y_train_prob)
train_precision, train_recall, _ = precision_recall_curve(y_train_res, y_train_prob)
train_pr_auc = auc(train_recall, train_precision)

print("\nTraining Set Evaluation:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"AUC-ROC Score: {train_roc_auc:.4f}")
print(f"Precision-Recall AUC: {train_pr_auc:.4f}")


# Evaluate Model Performance - Testing Set
test_accuracy = accuracy_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, y_test_prob)
test_precision, test_recall, _ = precision_recall_curve(y_test, y_test_prob)
test_pr_auc = auc(test_recall, test_precision)

print("\nTesting Set Evaluation:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"AUC-ROC Score: {test_roc_auc:.4f}")
print(f"Precision-Recall AUC: {test_pr_auc:.4f}")
print("\nTesting Classification Report:\n", classification_report(y_test, y_test_pred))

Fitting 5 folds for each of 10 candidates, totalling 50 fits

Best Hyperparameters: {'min_samples_split': 10, 'min_samples_leaf': 10, 'max_depth': 20, 'criterion': 'entropy'}

Training Set Evaluation:
Accuracy: 0.9987
AUC-ROC Score: 1.0000
Precision-Recall AUC: 1.0000

Testing Set Evaluation:
Accuracy: 0.9969
AUC-ROC Score: 0.9229
Precision-Recall AUC: 0.6421

Testing Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56654
           1       0.33      0.85      0.47        92

    accuracy                           1.00     56746
   macro avg       0.66      0.92      0.73     56746
weighted avg       1.00      1.00      1.00     56746



In [ ]:
'''
The Decision Tree model achieves near-perfect accuracy (0.9987) and AUC-ROC (1.0000) on the training set, indicating potential overfitting.
On the test set, accuracy remains high (0.9969), but AUC-ROC drops to 0.9229, meaning the model is not generalizing as well.
The recall for fraud cases (class 1) is high (0.85), indicating that the model detects most fraudulent transactions.
However, precision for fraud cases is low (0.33), suggesting a high number of false positives,
meaning many legitimate transactions are misclassified as fraud.
The Precision-Recall AUC of 0.6421 suggests moderate effectiveness in distinguishing fraud cases,
but the high discrepancy between training and testing performance confirms overfitting.
'''

In [33]:
# Importing important libraries
from imblearn.over_sampling import ADASYN

# ADASYN Sampling
adasyn = ADASYN(random_state = 5)
X_train_res, y_train_res = adasyn.fit_resample(X_train, y_train)

print('Before ADASYN, count of label 1 :', sum(y_train == 1))
print('Before ADASYN, count of label 0 :', sum(y_train == 0))

print('\nAfter ADASYN, count of label 1 :', sum(y_train_res == 1))
print('After ADASYN, count of label 0 :', sum(y_train_res == 0))

Before ADASYN, count of label 1 : 381
Before ADASYN, count of label 0 : 226599

After ADASYN, count of label 1 : 226630
After ADASYN, count of label 0 : 226599


In [35]:
# Hyperparameter Tuning using RandomizedSearchCV
param_dist = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10]
}

random_search = RandomizedSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

random_search.fit(X_train_res, y_train_res)

# Get best parameters
best_params = random_search.best_params_
print("\nBest Hyperparameters:", best_params)

# Train Optimized Model
model = DecisionTreeClassifier(**best_params, random_state=42)
model.fit(X_train_res, y_train_res)

# Predictions for Training Set
y_train_pred = model.predict(X_train_res)
y_train_prob = model.predict_proba(X_train_res)[:, 1]

# Predictions for Testing Set
y_test_pred = model.predict(X_test)
y_test_prob = model.predict_proba(X_test)[:, 1]

# Evaluate Model Performance - Training Set
train_accuracy = accuracy_score(y_train_res, y_train_pred)
train_roc_auc = roc_auc_score(y_train_res, y_train_prob)
train_precision, train_recall, _ = precision_recall_curve(y_train_res, y_train_prob)
train_pr_auc = auc(train_recall, train_precision)

print("\nTraining Set Evaluation:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"AUC-ROC Score: {train_roc_auc:.4f}")
print(f"Precision-Recall AUC: {train_pr_auc:.4f}")


# Evaluate Model Performance - Testing Set
test_accuracy = accuracy_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, y_test_prob)
test_precision, test_recall, _ = precision_recall_curve(y_test, y_test_prob)
test_pr_auc = auc(test_recall, test_precision)

print("\nTesting Set Evaluation:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"AUC-ROC Score: {test_roc_auc:.4f}")
print(f"Precision-Recall AUC: {test_pr_auc:.4f}")
print("\nTesting Classification Report:\n", classification_report(y_test, y_test_pred))

Fitting 5 folds for each of 10 candidates, totalling 50 fits

Best Hyperparameters: {'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 10, 'criterion': 'entropy'}

Training Set Evaluation:
Accuracy: 0.9780
AUC-ROC Score: 0.9944
Precision-Recall AUC: 0.9937

Testing Set Evaluation:
Accuracy: 0.9624
AUC-ROC Score: 0.9310
Precision-Recall AUC: 0.6189

Testing Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.96      0.98     56654
           1       0.04      0.86      0.07        92

    accuracy                           0.96     56746
   macro avg       0.52      0.91      0.52     56746
weighted avg       1.00      0.96      0.98     56746



In [ ]:
'''
Training Set Performance: The model performs well on the training set with accuracy of 97.80%, AUC-ROC of 0.9944,
and Precision-Recall AUC of 0.9937, indicating it has learned the patterns in the data effectively.
Testing Set Performance: The test accuracy is 96.24%, but the AUC-ROC drops to 0.9310, showing a decline in generalization.
Class Imbalance Issue: Precision for fraud cases (class 1) is only 0.04, 
meaning many non-fraud transactions are incorrectly flagged as fraud. 
However, recall is 0.86, which suggests the model detects a large portion of fraudulent cases.
Precision-Recall Tradeoff: The F1-score for fraud is just 0.07, 
indicating an imbalance between precision and recall—too many false positives (misclassified legitimate transactions).
Overfitting Concern: The difference in AUC-ROC (0.9944 in training vs. 0.9310 in testing)
and drop in precision suggest that the model is slightly overfitting.
'''